## Import libraries

In [ ]:
!pip install langchain

In [ ]:
!pip install -U pypdf
!pip install -U pypdf2
!pip install -U langchain
!pip install -U langchain-experimental
!pip install -U langchain-community
!pip install -U langchain-pinecone
!pip install -U sentence-transformers
!pip install -U pinecone-client
!pip install -U openai
!pip install -U pandas
!pip install -U numpy
!pip install -U scikit-learn
!pip install -U matplotlib
!pip install -U seaborn
!pip install -U gunicorn
!pip install -U python-dotenv
!pip install -U flask-cors
!pip install -U requests
!pip install -U openai
!pip install -U python-dotenv
!pip install -U flask-cors
!pip install -U requests

from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter


## Let extract the data from the pdf file using fucntions

In [4]:

def load_data(data):
    loader = DirectoryLoader(data,
                            glob='*.pdf',
                            loader_cls=PyPDFLoader)
    documents = loader.load()
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    docs = text_splitter.split_documents(documents)
    return docs

# Extract the content from the pdf file

In [ ]:


extracted_data = load_data(data='/Users/kwamebaffoe/Desktop/End-End-Generative-AI-Medical-Chatbot/Data')

In [70]:
# extracted_data

# Let Split the data into chunks using fucntions
def split_data(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    text_chunks = text_splitter.split_documents(extracted_data)
    return text_chunks


In [ ]:
text_chunks = split_data(extracted_data)
print(len(text_chunks))

# Let download the embeddings from Huggingface

In [74]:

from langchain.embeddings import HuggingFaceEmbeddings

In [6]:
from langchain.embeddings import HuggingFaceEmbeddings  

def download_huggingface_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [ ]:
# !pip install sentence-transformers
embeddings = download_huggingface_embeddings()


#  Let's convert to Vector Embeddings 

In [8]:
from langchain.embeddings import HuggingFaceEmbeddings  # Ensure you import this

def download_huggingface_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

# Call the function to get the embeddings
embeddings = download_huggingface_embeddings()

In [ ]:
# TEST CODES

from sentence_transformers import SentenceTransformer
from huggingface_hub import cached_download

model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")
print("Model loaded successfully!")


In [ ]:
# Now you can use the embeddings to embed the query
# !pip install sentence-transformers
import sentence_transformers

query_output = embeddings.embed_query("What is a heart attack?")
print("Length of query output: ", len(query_output))

In [ ]:
# query_output

# !pip install google
# !pip install "sentence-transformers>=3.0.0"
# !pip install sentence-transformers
# !pip install "torch>=1.11.0"

# !pip install torch==1.11.0
# !pip install sentence-transformers

# Example: Install torch with a specified CUDA or CPU version
# !pip install torch==1.11.0+cpu -f https://download.pytorch.org/whl/cpu/torch_stable.html


# from sentence_transformers import SentenceTransformer
# from huggingface_hub import cached_download

# !pip install sentence-transformers==2.2.2 huggingface-hub==0.12.0


# Creating Index with Pinecone Using Code Approach


In [ ]:
from dotenv import load_dotenv

load_dotenv()

In [12]:
import os
PINECONE_API_KEY = os.environ.get("PINECONE_API_KEY")

In [ ]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import SeverlessSpec
import os


#  Initialize Pinecone
pc = Pinecone(api_key="PINECONE_API_KEY"),
        
index = "einseinnmedicalchatbot"

pc.create_index(name="index_name",
                dimension=384,
                metric="cosine",
                spec=SeverlessSpec(),
                cloud="aws", 
                region="us-east-1",
            )

## Let create the Index with Python Codes in Simple Way


In [ ]:
from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(api_key="pcsk_2jGdBi_RFTHxF67eEHmau3ecngztXbgM8EGVQ1et1SMAJtom2t99yvUUZTWtRYGYAT4Gpd")

index_name = "einseinnmedicalchatbot"


pc.create_index(
    name=index_name,
    dimension=384, 
    metric="cosine",
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    ) 
)

# !pip install --upgrade pinecone-client


## CONVERT THE CHUNKS INTO VECTORS EMBEDDINGS

In [26]:
import os
from dotenv import load_dotenv

load_dotenv()

PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")

OPENAI_API_KEY = os.environ.get("PINECONE_API_KEY")

In [15]:
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY

In [ ]:
from langchain.vectorstores import Pinecone

docs_searching = Pinecone.from_documents(
    documents=text_chunks,
    embedding=embeddings,
    index_name="einseinnmedicalchatbot"  # Note: this should be a string
)

## Let load an exisiting Index 

In [17]:
from langchain.vectorstores import Pinecone

docs_searching = Pinecone.from_existing_index(
    embedding=embeddings,
    index_name="einseinnmedicalchatbot",
)

In [ ]:
docs_searching

In [ ]:
docs_searching.similarity_search("What is a heart attack?", k=2)

In [ ]:
retrieved_docs = docs_searching.similarity_search("What is a heart attack?", k=2)
retrieved_docs

In [22]:
retriever = docs_searching.as_retriever(search_type="similarity", search_kwargs={"k": 3})

In [23]:
retriever_docs = retriever.invoke( "What is an Acne?")

In [ ]:
retriever_docs

## Let make Sure the Query Goes to the Knowledge Base Model before it goes to the LLM

In [29]:
# using OpenAI LLM model for this
from langchain_openai import OpenAI

llm = OpenAI(temperature=0.4, max_tokens=500)


In [46]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_messages([
    ("system", 
     "You are a helpful assistant for question and answering task about medicine. "
     "Use the following pieces of retrieved context to answer the question. "
     "Say No to questions that are not about medicine. "
     "If you don't know the answer, just say that you don't know, don't try to make up an answer. "
     "Use three sentences maximum and keep the answer as short as possible.\n\n"
     "{context}"
    ),
    ("human", "{input}")
])



In [60]:
question_and_answer = create_stuff_documents_chain(
    llm=llm,
    prompt=prompt_template
)
rag_chain = create_retrieval_chain(
    retriever,
    question_and_answer
)

In [ ]:
response = rag_chain.invoke({"question": "What is acne?"})
print(response["answer"])


In [54]:
# Create the document chain
document_chain = create_stuff_documents_chain(
    llm=llm,
    prompt=prompt_template
)

# Create the retrieval chain
retrieval_chain = create_retrieval_chain(
    retriever=retriever,
    combine_docs_chain=document_chain
)

## Alternative Way

In [ ]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

# Create the prompt template
prompt = ChatPromptTemplate.from_messages([
    ("system", 
     "You are a helpful assistant for question and answering tasks about medicine. "
     "Use the following pieces of retrieved context to answer the question. "
     "If you don't know the answer, just say that you don't know.\n\n"
     "Context: {context}\n"
    ),
    ("human", "{input}")
])

# Create the document chain
document_chain = create_stuff_documents_chain(
    llm,
    prompt
)

# Create the retrieval chain
rag_chain = create_retrieval_chain(
    retriever,
    document_chain
)

# Test the chain
response = rag_chain.invoke({"input": "What is acne?"})
print(response["answer"])

## Let Create a ChatBot Using LangChain 



